# Sample Code for Google Colab

## Setting
install modules  
import modules  
define function and paths.  

In [1]:
%%bash
# clone our repository.
git clone https://github.com/ou-medinfo/medbertjp.git

# install transformers
pip install transformers

# install a Japanese tokenizer and morphological analysis tools.
apt-get install mecab
pip install fugashi ipadic unidic
python -m unidic download

  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b09d5d22a558d9ddd3c696dc21d220f8edec9bbcd4b2dee2d45c39ae67ab2e2e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libmecab2 mecab-jumandic mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  libmecab2 mecab mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 5 newly installed, 0 to remove and 21 not upgraded.
Need to get 16.5 MB of archives.
After this operation, 219 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:3 http://archive.ubuntu.com/ubuntu bi

Cloning into 'medbertjp'...
unidic.zip: 100%|██████████| 608M/608M [00:35<00:00, 17.3MB/s]
Downloaded UniDic v2.3.0+2020-10-08 to /usr/local/lib/python3.6/dist-packages/unidic/dicdir


In [2]:
import random
import torch
from medbertjp.tokenization_jp_mod import BertJapaneseTokenizer
from transformers.modeling_bert import BertForMaskedLM

In [3]:
def prediction_masked_sentence(sentence, tokenizer, model):
    mask_token_id = tokenizer.mask_token_id
    
    input_ids = tokenizer.encode(text, return_tensors='pt')
    print(' '.join(tokenizer.convert_ids_to_tokens(input_ids.tolist()[0])))
    masked_index = random.randint(1, len(input_ids[0]) - 2)
    input_ids[0][masked_index] = mask_token_id
    print(' '.join(tokenizer.convert_ids_to_tokens(input_ids.tolist()[0])))
    
    result = model(input_ids)
    topk = torch.nn.Softmax(dim=1)(result[0][:, masked_index]).topk(5)
    scores = topk.values.tolist()[0]
    pred_ids = topk.indices.tolist()[0]
    for i, pred_id in enumerate(pred_ids):
        output_ids = input_ids.tolist()[0]
        output_ids[masked_index] = pred_id
        
        pred_sentence = tokenizer.convert_ids_to_tokens(output_ids)
        pred_sentence = [token if n != masked_index else '【' + token + '】'
                         for n, token in enumerate(pred_sentence)]
        print('{:.1f} %'.format(scores[i] * 100).rjust(7), ' '.join(pred_sentence))
        
    return result

In [4]:
text = '慢性膵炎では，糖尿病性ケトアシドーシス(DKA)の合併は比較的稀である．'

# Model path
PathToBertJpMecabUnidic = './medBERTjp_L12_H768_A12_WWM_mecab-unidic'
PathToBertJpMecabIpadic = './medBERTjp_L12_H768_A12_WWM_mecab-ipadic'
PathToBertJpSentencepiece = './medBERTjp_L12_H768_A12_sentencepiece'

##  MeCab-IPAdic-tokenized Japanese BERT model
For general sentences.

In [5]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking',
                                                  word_tokenizer_type='mecab',
                                                  mecab_kwargs={
                                                     "mecab_dic": 'ipadic',
                                                 })
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# An Example of Masked Language Modeling.
result = prediction_masked_sentence(text, tokenizer, model)

[CLS] 慢性 [UNK] で は , 糖尿 病 性 ケ ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
[CLS] 慢性 [UNK] で は , 糖尿 病 性 [MASK] ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 50.7 % [CLS] 慢性 [UNK] で は , 糖尿 病 性 【ケ】 ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  7.9 % [CLS] 慢性 [UNK] で は , 糖尿 病 性 【網】 ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  6.1 % [CLS] 慢性 [UNK] で は , 糖尿 病 性 【アセ】 ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  4.9 % [CLS] 慢性 [UNK] で は , 糖尿 病 性 【コリン】 ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  3.4 % [CLS] 慢性 [UNK] で は , 糖尿 病 性 【コレ】 ##ト ##アシ ##ドー ##シス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]


## medBERTjp - SentencePiece
For medical sentences.

In [ ]:
%%bash
wget https://github.com/ou-medinfo/medbertjp/releases/download/v0.1-sp/medBERTjp_L12_H768_A12_sentencepiece.zip
unzip ./medBERTjp_L12_H768_A12_sentencepiece.zip

In [8]:
model_path = PathToBertJpSentencepiece
tokenizer = BertJapaneseTokenizer.from_pretrained(model_path,
                                                  word_tokenizer_type='sp',
                                                 )
model = BertForMaskedLM.from_pretrained(model_path)

medBERTjp_L12_H768_A12_sentencepiece/vocab.model
Loaded a trained SentencePiece model.


Some weights of the model checkpoint at ./medBERTjp_L12_H768_A12_sentencepiece were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# An Example of Masked Language Modeling.
result = prediction_masked_sentence(text, tokenizer, model)

[CLS] ▁ 慢性膵炎 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 稀である . [SEP]
[CLS] ▁ 慢性膵炎 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 [MASK] . [SEP]
 39.1 % [CLS] ▁ 慢性膵炎 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 【少ない】 . [SEP]
 24.5 % [CLS] ▁ 慢性膵炎 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 【稀である】 . [SEP]
 15.2 % [CLS] ▁ 慢性膵炎 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 【多い】 . [SEP]
  8.7 % [CLS] ▁ 慢性膵炎 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 【高い】 . [SEP]
  3.4 % [CLS] ▁ 慢性膵炎 では , 糖尿病性ケトアシドーシス ( D KA ) の合併 は比較的 【まれである】 . [SEP]


## medBERTjp - MeCab with IPAdic
For medical sentences.

In [ ]:
%%bash
wget https://github.com/ou-medinfo/medbertjp/releases/download/v0.1-mi/medBERTjp_L12_H768_A12_WWM_mecab-ipadic.zip
unzip ./medBERTjp_L12_H768_A12_WWM_mecab-ipadic.zip

In [11]:
model_path = PathToBertJpMecabIpadic
tokenizer = BertJapaneseTokenizer.from_pretrained(model_path, 
                                                 mecab_kwargs={
                                                     "mecab_dic": 'ipadic',
                                                 })
model = BertForMaskedLM.from_pretrained(model_path)

Some weights of the model checkpoint at ./medBERTjp_L12_H768_A12_WWM_mecab-ipadic were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# An Example of Masked Language Modeling.
result = prediction_masked_sentence(text, tokenizer, model)

[CLS] 慢性 膵炎 で は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
[CLS] 慢性 膵炎 [MASK] は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 81.3 % [CLS] 慢性 膵炎 【で】 は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
 11.3 % [CLS] 慢性 膵炎 【において】 は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  1.9 % [CLS] 慢性 膵炎 【に関して】 は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  1.8 % [CLS] 慢性 膵炎 【に】 は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]
  0.8 % [CLS] 慢性 膵炎 【以外】 は , 糖尿 病 性 ケトアシドーシス ( D ##KA ) の 合併 は 比較的 稀 で ある . [SEP]


## medBERTjp - MeCab with Unidic
is not available now, but will be uploaded soon.


In [13]:
# model_path = PathToBertJpMecabUnidic
# tokenizer = BertJapaneseTokenizer.from_pretrained(model_path, 
#                                                  mecab_kwargs={
#                                                      "mecab_dic": 'unidic',
#                                                  })
# model = BertForMaskedLM.from_pretrained(model_path)

In [14]:
# # An Example of Masked Language Modeling.
# result = prediction_masked_sentence(text, tokenizer, model)